In [3]:
#| default_exp utils

In [2]:
#| hide
from nbdev.showdoc import *

In [40]:
#| export
from loguru import logger
from functools import wraps
import mmcv
import inspect
ICACHE = dict()
#export
import xxhash
import pickle
import mmcv

def identify(x):
    '''Return an hex digest of the input'''
    return xxhash.xxh64(pickle.dumps(x), seed=0).hexdigest()
VERBOSE = False
CACHE_DIR = '/tmp/avcv/'
def memoize(func):
    import os
    import pickle
    from functools import wraps
    import xxhash
    '''Cache result of function call on disk
    Support multiple positional and keyword arguments'''
    @wraps(func)
    def memoized_func(*args, **kwargs):
        try:
            import inspect
            if 'cache_key' in kwargs:
                cache_key = kwargs['cache_key']
                
                func_id = identify(kwargs['cache_key'])
                if VERBOSE:
                    logger.info(f'Use {cache_key=}->{func_id=}')
            else:
                func_id = identify((inspect.getsource(func), args, kwargs))
            cache_path = os.path.join(CACHE_DIR, 'funcs', func.__name__+'/'+func_id)
            mmcv.mkdir_or_exist(os.path.dirname(cache_path))

            if (os.path.exists(cache_path) and
                    not func.__name__ in os.environ and
                    not 'BUST_CACHE' in os.environ):
                result = pickle.load(open(cache_path, 'rb'))
            else:
                result = func(*args, **kwargs)
                pickle.dump(result, open(cache_path, 'wb'))
            return result
        except (KeyError, AttributeError, TypeError, Exception) as e:
            logger.warning(f'Exception: {e}, use default function call')
            return func(*args, **kwargs)
    return memoized_func

def imemoize(func):
    """
        Memoize a function into memory, the function recaculate only 
        change when its belonging arguments change
    """
    @wraps(func)
    def _f(*args, **kwargs):

        timer = mmcv.Timer()

        ident_name = identify((inspect.getsource(func), args, kwargs))
        # if not ident_name in ICACHE:
        try:
            result = ICACHE[ident_name]
        except:
            result = func(*args, **kwargs)
            ICACHE[ident_name] = result
        return result
    return _f

In [48]:
@memoize
def example_function1(x, cache_key=None):
    print('Heelo')
    return 1
x = example_function1(0, cache_key='2')

# BUILD

In [49]:
!nbdev_build_lib

Converted 00_visualize.ipynb.
Converted 01_process.ipynb.
Converted 03_memoize.ipynb.
Converted 03_utils.ipynb.
Converted 04_debug.ipynb.
Converted 05_coco_dataset.ipynb.
Converted 06_cli.ipynb.
Converted 07_dist_utils.ipynb.
Converted all.ipynb.
Converted index.ipynb.
